In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('Final_cleaned.csv')

In [3]:
df.shape

(1070994, 12)

In [6]:
null_zero = pd.DataFrame(dict(zip(['# of null', '# of zeros'], [df.isna().sum(), (df == 0).sum()])))
null_zero

,# of null,# of zeros
RECORD,0,0
B,0,0
TAXCLASS,0,0
ZIP,0,0
LTFRONT,0,0
LTDEPTH,0,0
STORIES,0,0
BLDFRONT,0,0
BLDDEPTH,0,0
FULLVAL,0,0


In [7]:
df['ZIP3'] = df.ZIP.apply(lambda x: int(str(x)[:3]))

In [8]:
df.head()

,RECORD,B,TAXCLASS,ZIP,LTFRONT,LTDEPTH,STORIES,BLDFRONT,BLDDEPTH,FULLVAL,AVLAND,AVTOT,ZIP3
0,1,1,4,10004.0,500.0,1046.0,50.0,152.0,207.0,21400000.0,4225500.0,9630000.0,100
1,2,1,4,10004.0,27.0,207.0,50.0,152.0,207.0,193800000.0,14310000.0,87210000.0,100
2,3,1,4,10004.0,709.0,564.0,3.0,709.0,564.0,104686000.0,39008700.0,47108700.0,100
3,4,1,4,10004.0,793.0,551.0,2.0,85.0,551.0,39200000.0,15255000.0,17640000.0,100
4,5,1,4,10004.0,323.0,1260.0,1.0,89.0,57.0,272300000.0,121050000.0,122535000.0,100


In [9]:
null_zero = pd.DataFrame(dict(zip(['# of null', '# of zeros'], [df.isna().sum(), (df == 0).sum()])))
null_zero

,# of null,# of zeros
RECORD,0,0
B,0,0
TAXCLASS,0,0
ZIP,0,0
LTFRONT,0,0
LTDEPTH,0,0
STORIES,0,0
BLDFRONT,0,0
BLDDEPTH,0,0
FULLVAL,0,0


In [14]:
def ratios(fullval, avland, avtot, ltf, ltd, blf, bld, sto):
    
    s1 = ltf * ltd
    s2 = blf * bld
    s3 = s2 * sto
    
    return pd.concat([pd.Series(value/ratio) for value in [fullval, avland, avtot] for ratio in [s1,s2,s3]], axis = 1)

In [12]:
import timeit

In [15]:
start = timeit.default_timer()

df[['r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'r9']] = ratios(df.FULLVAL.values, df.AVLAND.values,df.AVTOT.values,
                                                                       df.LTFRONT.values,df.LTDEPTH.values,
                                                                      df.BLDFRONT.values,df.BLDDEPTH.values, df.STORIES.values)

print('Total run time: {}s'.format(timeit.default_timer() - start))

Total run time: 0.24399536900000385


In [16]:
df.head()

,RECORD,B,TAXCLASS,ZIP,LTFRONT,LTDEPTH,STORIES,BLDFRONT,BLDDEPTH,FULLVAL,...,ZIP3,r1,r2,r3,r4,r5,r6,r7,r8,r9
0,1,1,4,10004.0,500.0,1046.0,50.0,152.0,207.0,21400000.0,...,100,40.917782,680.142385,13.602848,8.079350,134.296339,2.685927,18.413002,306.064073,6.121281
1,2,1,4,10004.0,27.0,207.0,50.0,152.0,207.0,193800000.0,...,100,34675.254965,6159.420290,123.188406,2560.386473,454.805492,9.096110,15603.864734,2771.739130,55.434783
2,3,1,4,10004.0,709.0,564.0,3.0,709.0,564.0,104686000.0,...,100,261.796157,261.796157,87.265386,97.551991,97.551991,32.517330,117.808271,117.808271,39.269424
3,4,1,4,10004.0,793.0,551.0,2.0,85.0,551.0,39200000.0,...,100,89.714219,836.980890,418.490445,34.913021,325.717946,162.858973,40.371399,376.641401,188.320700
4,5,1,4,10004.0,323.0,1260.0,1.0,89.0,57.0,272300000.0,...,100,669.074647,53676.325646,53676.325646,297.434763,23861.620343,23861.620343,301.083591,24154.346541,24154.346541


In [17]:
def more_ratios(i,x):
    
    z5 = x[0]
    z3 = x[1]
    tc = x[2]
    boro = x[3]
    ra = x[4]
    
    means = [df[df.ZIP == z5]['r'+str(i)].mean(),
                    df[df.ZIP3 == z3]['r'+str(i)].mean(),
                    df[df.TAXCLASS == tc]['r'+str(i)].mean(),
                    df[df.B == boro]['r'+str(i)].mean(),
                    df['r'+str(i)].mean()]
    
    return [round(ra/mean, 4) for mean in means]

In [18]:
dic = {1: 'r1_', 2:'r2_', 3:'r3_', 4:'r4_', 5:'r5_', 6: 'r6_', 7:'r7_', 8:'r8_', 9:'r9_'}

In [19]:
start = timeit.default_timer()

for i in range(1,10):
    df[[dic[i]+'1', dic[i]+'2', dic[i]+'3', dic[i]+'4', dic[i]+'5']] \
                = df[['ZIP', 'ZIP3', 'TAXCLASS', 'B', 'r'+str(i)]].apply(lambda x: pd.Series(more_ratios(i, x)), axis = 1)

print('Total run time: {}s'.format(round(timeit.default_timer() - start),3))

KeyboardInterrupt: 